In [0]:
!pip install -q tensorflow==2.0.0-beta1

     |████████████████████████████████| 87.9MB 1.1MB/s 
     |████████████████████████████████| 3.1MB 18.2MB/s 
     |████████████████████████████████| 501kB 59.0MB/s 


In [0]:
import tensorflow as tf
print(tf.__version__)

2.0.0-beta1


In [0]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model

In [0]:

cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = y_train.flatten(), y_test.flatten() ##
print("x_train.shape:", x_train.shape)
print("y_train.shape", y_train.shape)

170500096/170498071 [==============================] - 6s 0us/step
x_train.shape: (50000, 32, 32, 3)
y_train.shape (50000,)


In [0]:
K = len(set(y_train))
print("No. of classes "+str(K))

10

In [0]:
i = Input(shape = x_train[0].shape)

x = Conv2D(32, (3,3), padding = 'same', activation = 'relu')(i)
x = BatchNormalization()(x)
x = Conv2D(32, (3,3), padding = 'same', activation = 'relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(64, (3,3), padding = 'same', activation = 'relu')(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3,3), padding = 'same', activation = 'relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(128, (3,3), padding = 'same', activation = 'relu')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3,3), padding = 'same', activation = 'relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)

x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(1024, activation = 'relu')(x)
x = Dropout(0.2)(x)
x = Dense(K, activation = 'softmax')(x)

model = Model(i,x)

In [0]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
r = model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=15)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 50000 samples, validate on 10000 samples
Epoch 1/15
50000/50000 [==============================] - 469s 9ms/sample - loss: 1.2759 - accuracy: 0.5609 - val_loss: 1.0277 - val_accuracy: 0.6451
Epoch 2/15
50000/50000 [==============================] - 451s 9ms/sample - loss: 0.8388 - accuracy: 0.7084 - val_loss: 0.8920 - val_accuracy: 0.6933
Epoch 3/15
37760/50000 [=====================>........] - ETA: 1:51 - loss: 0.6855 - accuracy: 0.7627

KeyboardInterrupt: ignored

In [0]:
# Fit with data augmentation
# Note: if you run this AFTER calling the previous model.fit(), it will CONTINUE training where it left off
batch_size = 32
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(width_shift_range=0.1, height_shift_range = 0.1, horizontal_flip = True)
train_generator = data_generator.flow(x_train,y_train,batch_size)
steps_per_epoch = x_train.shape[0] // batch_size
r = model.fit_generator(train_generator, validation_data=(x_test, y_test), steps_per_epoch=steps_per_epoch, epochs=50)

Epoch 1/50
1562/1562 [==============================] - 527s 338ms/step - loss: 0.5554 - accuracy: 0.8120 - val_loss: 0.7270 - val_accuracy: 0.7669
Epoch 2/50
1562/1562 [==============================] - 505s 324ms/step - loss: 0.5249 - accuracy: 0.8211 - val_loss: 0.6089 - val_accuracy: 0.7983
Epoch 3/50
1562/1562 [==============================] - 468s 299ms/step - loss: 0.4944 - accuracy: 0.8319 - val_loss: 0.5017 - val_accuracy: 0.8307
Epoch 4/50
1562/1562 [==============================] - 461s 295ms/step - loss: 0.4673 - accuracy: 0.8402 - val_loss: 0.4891 - val_accuracy: 0.8344
Epoch 5/50
1562/1562 [==============================] - 463s 297ms/step - loss: 0.4482 - accuracy: 0.8470 - val_loss: 0.5700 - val_accuracy: 0.8212
Epoch 6/50
1562/1562 [==============================] - 466s 298ms/step - loss: 0.4251 - accuracy: 0.8550 - val_loss: 0.4569 - val_accuracy: 0.8436
Epoch 7/50
1562/1562 [==============================] - 463s 296ms/step - loss: 0.4137 - accuracy: 0.8600 - val_

In [0]:
import matplotlib.pyplot as plt
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()

NameError: ignored

In [0]:
plt.plot(r.history['accuracy'], label='acc')
plt.plot(r.history['val_accuracy'], label='val_acc')
plt.legend()

In [0]:
# Plot confusion matrix
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
  """
  This function prints and plots the confusion matrix.
  Normalization can be applied by setting `normalize=True`.
  """
  if normalize:
      cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
      print("Normalized confusion matrix")
  else:
      print('Confusion matrix, without normalization')

  print(cm)

  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)

  fmt = '.2f' if normalize else 'd'
  thresh = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      plt.text(j, i, format(cm[i, j], fmt),
               horizontalalignment="center",
               color="white" if cm[i, j] > thresh else "black")

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.show()


p_test = model.predict(x_test).argmax(axis=1)
cm = confusion_matrix(y_test, p_test)
plot_confusion_matrix(cm, list(range(10)))



In [0]:
# label mapping
labels = '''airplane
automobile
bird
cat
deer
dog
frog
horse
ship
truck'''.split()

In [0]:
misclassified_idx = np.where(p_test != y_test)[0]
i = np.random.choice(misclassified_idx)
plt.imshow(x_test[i], cmap='gray')
plt.title("True label: %s Predicted: %s" % (labels[y_test[i]], labels[p_test[i]]));

In [0]:
model.summary()